# Sesi 15 / Model Deployment




## Importing Libraries



Notes: Profiling boleh diinstall atau tidak diinstall. Dalam sesi ini, profiling hanya digunakan menjadi contoh untuk memudahkan analisa.

Lewati cell report = data.profile_report(title='Campus Placement Data - Report', progress_bar=False) Jika memutuskan untuk tidak menggunakan profiling.



In [1]:
!pip install pandas-profiling

  Using cached MarkupSafe-2.1.1-cp39-cp39-win_amd64.whl (17 kB)

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\RIDHO\\anaconda3\\Lib\\site-packages\\~~rkupsafe\\_speedups.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import pickle

%matplotlib inline

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\RIDHO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\RIDHO\AppData\Local\Temp\ipykernel_23372\1120705230.py", line 5, in <cell line: 5>
    import pandas_profiling
  File "C:\Users\RIDHO\anaconda3\lib\site-packages\pandas_profiling\__init__.py", line 6, in <module>
    from pandas_profiling.controller import pandas_decorator
  File "C:\Users\RIDHO\anaconda3\lib\site-packages\pandas_profiling\controller\pandas_decorator.py", line 4, in <module>
    from pandas_profiling.profile_report import ProfileReport
  File "C:\Users\RIDHO\anaconda3\lib\site-packages\pandas_profiling\profile_report.py", line 27, in <module>
    from pandas_profiling.report.presentation.flavours.html.templates import (
  File "C:\Users\RIDHO\anaconda3\lib\site-packages\pandas_profiling\report\presentation\flavours\html\__init__.py", line 1, in <module>
    

In [ ]:
data = pd.read_csv('Placement_Data_Full_Class.csv')

#Remove Serial Number
data.drop("sl_no", axis=1, inplace=True)

### Examining Dataset


In [ ]:
data.head()

Kesimpulan

- Kami memiliki data kualifikasi Gender dan Pendidikan
- Kami memiliki semua data kinerja (skor) pendidikan
- Kami memiliki status penempatan dan detail gaji
- Kita dapat mengharapkan nilai nol dalam gaji karena kandidat yang tidak ditempatkan tidak akan memiliki gaji
- Status penempatan adalah variabel target kami sisanya adalah variabel independen kecuali gaji

In [ ]:
print ("The shape of the  data is (row, column):"+ str(data.shape))
print (data.info())

In [ ]:
report = data.profile_report(title='Campus Placement Data - Report',progress_bar=False)
report

# Uncomment syntax below to make new html file

# report.to_file(output_file='output.html')

- 67 Nilai yang hilang dalam Gaji untuk siswa yang tidak ditempatkan. Nilai NaN perlu diisi.
- Data tidak diskalakan. Kolom gaji berkisar antara 200k-940k, kolom numerik lainnya adalah persentase.
- 300k pada Persentil ke-75 berjalan hingga maks 940k, dalam Gaji (skewnwss tinggi). Dengan demikian, outlier pada akhir gaji tinggi.

Data Cleaning / Handling missing values


In [ ]:
data['salary'].fillna(value=0, inplace=True)
print('Salary column with null values:',
      data['salary'].isnull().sum(), sep = '\n')

Exploring Data by each Features (Optional)


In [ ]:
"""
Feature: Gender

Does gender affect placements?
"""

data.gender.value_counts()

In [ ]:
sns.countplot("gender", hue="status", data=data)
plt.show()

In [ ]:
sns.kdeplot(data.salary[data.gender=="M"])
sns.kdeplot(data.salary[data.gender=="F"])
plt.legend(["Male", "Female"])
plt.xlabel("Salary (100k)")
plt.show()

In [ ]:
plt.figure(figsize =(18,6))
sns.boxplot("salary", "gender", data=data)
plt.show()

Wawasan

- Kami memiliki sampel 139 siswa laki-laki dan 76 siswa perempuan.
- 30 siswa perempuan dan 40 siswa laki-laki tidak ditempatkan. - - Siswa laki-laki memiliki penempatan yang relatif lebih tinggi.
- Siswa laki-laki ditawari gaji yang sedi

In [ ]:
"""
Feature: ssc_p (Secondary Education percentage), ssc_b (Board Of Education)

Does Secondary Education affect placements?
"""
sns.kdeplot(data.ssc_p[data.status=="Placed"])
sns.kdeplot(data.ssc_p[data.status=="Not Placed"])
plt.legend(["Placed", "Not Placed"])
plt.xlabel("Secondary Education Percentage")
plt.show()

- Semua siswa dengan Persentase Pendidikan Menengah di atas 90% ditempatkan
- Semua siswa dengan Persentase Pendidikan Menengah di bawah 50% tidak ditempatkan
- Siswa dengan Persentase Pendidikan Menengah yang baik ditempatkan rata-rata.

In [ ]:
sns.countplot("ssc_b", hue="status", data=data)
plt.show()


Board Of Education tidak banyak mempengaruhi Placement Status.



In [ ]:
plt.figure(figsize =(18,6))
sns.boxplot("salary", "ssc_b", data=data)
plt.show()

Outliers terdapat pada keduanya, tetapi siswa dari Central Board mendapatkan pekerjaan bergaji tinggi.



In [ ]:
sns.lineplot("ssc_p", "salary", hue="ssc_b", data=data)
plt.show()

Tidak ada pola khusus (correlation) antara Secondary Education Percentage dan Salary. Board of Education Tidak Mempengaruhi Gaji

Kita dapat membuat grafik sendiri dari sini.

In [ ]:
sns.set(rc={'figure.figsize':(12,8)})
sns.set(style="white", color_codes=True)
sns.jointplot(x=data["etest_p"], y=data["salary"], kind='kde', color="skyblue")

plt.show()

- Sebagian besar kandidat yang mendapat skor sekitar 60 persen mendapat paket yang layak sekitar 3 lakh PA
- Tidak banyak kandidat yang menerima gaji lebih dari 4 lakh PA
- Bagian padat bawah menunjukkan kandidat yang tidak ditempatkan

In [ ]:
plt.figure(figsize = (15, 7))
plt.style.use('seaborn-white')
plt.subplot(231)
sns.distplot(data['ssc_p'])
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.subplot(232)
sns.distplot(data['hsc_p'])
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.subplot(233)
sns.distplot(data['degree_p'])
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.subplot(234)
sns.distplot(data['etest_p'])
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.subplot(235)
sns.distplot(data['mba_p'])
fig = plt.gcf()
fig.set_size_inches(10,10)

plt.subplot(236)
sns.distplot(data['salary'])
fig = plt.gcf()
fig.set_size_inches(10,10)

- Semua distribusi mengikuti distribusi normal kecuali fitur gaji
- Sebagian besar kinerja pendidikan calon adalah antara 60-80%
- Distribusi gaji mendapat outlier dimana hanya sedikit yang mendapat gaji 7,5L dan 10L PA

In [ ]:
plt.style.use('seaborn-white')
f,ax=plt.subplots(1,2,figsize=(18,8))
data['workex'].value_counts().plot.pie(explode=[0,0.05], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Work experience')
sns.countplot(x = 'workex',hue = "status",data = data)
ax[1].set_title('Influence of experience on placement')

plt.show()

- Kami memiliki hampir 66,2% kandidat yang tidak pernah memiliki pengalaman kerja
- Kandidat yang tidak pernah memiliki pengalaman kerja lebih banyak dipekerjakan daripada yang memiliki pengalaman
- Kita dapat menyimpulkan bahwa pengalaman kerja tidak mempengaruhi kandidat dalam proses rekrutmen

## Preprocessing data for classfication models


In [ ]:
from sklearn.preprocessing import LabelEncoder

object_cols = ['gender','workex','specialisation','status']

label_encoder = LabelEncoder()
for col in object_cols:
    data[col] = label_encoder.fit_transform(data[col])
data.head()

In [ ]:
# Library imports
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Seperating Features and Target
X = data[['ssc_p', 'hsc_p', 'degree_p', 'etest_p']]
y = data['status']

In [ ]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

In [ ]:
# Decision Tree
dtree = DecisionTreeClassifier(criterion='entropy')
dtree.fit(X_train, y_train)

y_pred = dtree.predict(X_test)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#Using Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)

y_pred = random_forest.predict(X_test)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
#Using Logistic Regression

LogR = LogisticRegression(solver='lbfgs')
LogR.fit(X_train, y_train)

y_pred = LogR.predict(X_test)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Getting ready for deployment


## Pickle

Modul pickle Python digunakan untuk serializing dan de-serializing struktur objek Python. Objek apa pun dengan Python dapat dibuat pickled sehingga dapat disimpan di disk. Apa yang dilakukan pickle adalah 'serializes' objek terlebih dahulu sebelum menulisnya ke file. Pickling adalah cara untuk mengubah objek python (list, dict, etc.) Menjadi character stream. Idenya adalah bahwa character stream ini berisi semua informasi yang diperlukan untuk merekonstruksi objek dalam skrip python lain.



In [ ]:
pickle.dump(random_forest, open('model_classifier.pkl','wb'))

## Brief introduction to HTML and CSS


HTML adalah standard markup language untuk membuat halaman Web.



## What is HTML?
- HTML stands for Hyper Text Markup Language
- HTML is the standard markup language for creating Web pages
- HTML describes the structure of a Web page
- HTML consists of a series of elements
- HTML elements tell the browser how to display the content
- HTML elements label pieces of content such as "this is a heading", "this is a paragraph", "this is a link", etc.

## A Simple HTML Document


```
<!DOCTYPE html>
<html>
<head>
  <title>Page Title</title>
</head>
<body>

  <h1>My First Heading</h1>
  <p>My first paragraph.</p>

</body>
</html>
```

- ```<!DOCTYPE html>``` mendefinisikan bahwa dokumen ini adalah dokumen HTML5
- ```<html>``` elemen adalah elemen root dari halaman HTML
- ```<head>``` elemen berisi informasi meta tentang halaman HTML
- ```<title>``` elemen menentukan judul untuk halaman HTML (yang ditampilkan di judul browser atau di tab halaman)
- ```<body>``` elemen mendefinisikan badan dokumen, dan merupakan wadah untuk semua konten yang terlihat, seperti headings, paragraphs, images, hyperlinks, tables, lists, dll.
- ```<h1>``` elemen mendefinisikan headingbesar
- The ```<p>``` element mendefinisikan sebuah paragraph

## What is an HTML Element?


Elemen HTML ditentukan oleh tag awal, beberapa konten, dan tag akhir:

```<tagname>Content goes here...</tagname>```

Elemen HTML adalah apapun yang dimulai dari tag awal hingga tag akhir:

```
<h1>My First Heading</h1>
<p>My first paragraph.</p>
```

CSS adalah singkatan dari Cascading Style Sheets.

CSS menghemat banyak pekerjaan. CSS dapat mengontrol tata letak beberapa halaman web sekaligus.

## What is CSS?


Cascading Style Sheets (CSS) digunakan untuk memformat tata letak halaman web.

Dengan CSS, kita dapat mengontrol warna, font, ukuran teks, jarak antar elemen, bagaimana elemen diposisikan dan ditata, gambar latar belakang atau warna latar yang akan digunakan, tampilan yang berbeda untuk perangkat dan ukuran layar yang berbeda, dan banyak lagi.

## CSS Syntax


Set aturan CSS terdiri dari selector dan declaration block:



<img src="https://i.ibb.co/zbJTWJ3/css.gif" width="500" align="center">


- The selector points to the HTML element you want to style.
- The declaration block contains one or more declarations separated by semicolons.
- Each declaration includes a CSS property name and a value, separated by a colon.
- Multiple CSS declarations are separated with semicolons, and declaration blocks are surrounded by curly braces.

## Using CSS


CSS dapat ditambahkan ke dokumen HTML dengan 3 cara:

CSS dapat ditambahkan ke dokumen HTML dengan 3 cara:

- Inline - by using the style attribute inside HTML elements
- Internal - by using a ```<style>``` element in the ```<head>``` section
- External - by using a ```<link>``` element to link to an external CSS file

Cara paling umum untuk menambahkan CSS, adalah menempatkan style dalam file CSS eksternal.

## Inline CSS
Inline CSS digunakan untuk menerapkan style unik ke satu elemen HTML.

Inline CSS menggunakan style attribute dari elemen HTML.

Contoh berikut bagaimana cara mengatur warna teks elemen ```<h1>``` menjadi biru, dan warna teks elemen ```<p>`` menjadi merah:

```
<h1 style="color:blue;">A Blue Heading</h1>

<p style="color:red;">A red paragraph.</p>
```


## Internal CSS

CSS internal digunakan untuk menentukan style untuk satu halaman HTML.

CSS internal ditentukan di bagian <head> halaman HTML, dalam elemen <style>.

Contoh berikut mengatur warna teks SEMUA elemen <h1> (pada halaman itu) menjadi biru, dan warna teks SEMUA elemen <p> menjadi merah. Selain itu, halaman akan ditampilkan dengan warna latar belakang "powderblue":

```
<!DOCTYPE html>
<html>
<head>
<style>
  body {background-color: powderblue;}
  h1   {color: blue;}
  p    {color: red;}
</style>
</head>
<body>

  <h1>This is a heading</h1>
  <p>This is a paragraph.</p>

</body>
</html>
```

## External CSS
Style sheet eksternal digunakan untuk menentukan style untuk banyak halaman HTML.

Untuk menggunakan style sheet eksternal, tambahkan link ke dalamnya di bagian <head> pada setiap halaman HTML:

```
<!DOCTYPE html>
<html>
<head>
  <link rel="stylesheet" href="styles.css">
</head>
<body>

  <h1>This is a heading</h1>
  <p>This is a paragraph.</p>

</body>
</html>
```

Style sheet eksternal dapat ditulis di editor teks manapun. File tidak boleh berisi kode HTML apa pun, dan harus disimpan dengan ekstensi .css.

Berikut adalah tampilan file styles.css:

```
body {
  background-color: powderblue;
}
h1 {
  color: blue;
}
p {
  color: red;
}
```

# Creating web app with Flask

Pastikan flask sudah terinstall dengan ```pip install flask```.

Minimal Flask application terlihat seperti ini:

In [ ]:
!pip install flask

<b> Kemudian,kita membuat file python dengan nama file hello.py

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

Command dibawah ini akan meluncurkan server bawaan yang sangat sederhana, yang cukup baik untuk testing 

In [ ]:
$ export FLASK_APP=hello.py
$ flask run

Sekarang akses ke http://127.0.0.1:5000/ dan kita akan melihat hello world pada tampilan depan web flask yang telah kita buat.

# Debug Mode

Untuk mengaktifkan semua fitur development (termasuk mode debug) kita dapat mengekspor FLASK_ENV environment variable dan menyetelnya ke development sebelum menjalankan server:

In [ ]:
$ export FLASK_ENV=development
$ flask run

Code tersebut berguna untuk:
1. Mengaktifkan debugger
2. Mengaktifkan reloader otomatis
3. Mengaktifkan mode debug pada aplikasi Flask

Kita juga dapat mengontrol mode debug secara terpisah dari environment dengan mengekspor FLASK_DEBUG=1

# Routing

Gunakan dekorator route() untuk mengikat fungsi ke URL.

In [ ]:
@app.route('/')
def index():
    return 'Index Page'

@app.route('/hello')
def hello():
    return 'Hello, World'

# Variable Rules

Kita dapat menambahkan bagian variabel ke URL dengan menandai bagian dengan `<variable_name>`

In [ ]:
from markupsafe import escape

@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return 'User %s' % escape(username)

@app.route('/post/<int:post_id>')
def show_post(post_id):
    # show the post with the given id, the id is an integer
    return 'Post %d' % post_id

@app.route('/path/<path:subpath>')
def show_subpath(subpath):
    # show the subpath after /path/
    return 'Subpath %s' % escape(subpath)

## HTTP Methods


Aplikasi web menggunakan metode HTTP yang berbeda saat mengakses URL.Secara default,route hanya menjawab permintaan GET.Kita bisa menggunakan argumen metode dekorator route() untuk menangani metode HTTP yang berbeda.

In [ ]:
from flask import request

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return do_the_login()
    else:
        return show_the_login_form()

Jika GET tersedia, Flask secara otomatis menambahkan dukungan untuk metode HEAD dan menangani permintaan HEAD sesuai dengan HTTP RFC. Demikian juga, OPTIONS diterapkan secara otomatis untuk kita.

## Static Files

menghasilkan URL untuk file statis, gunakan endpoint name 'static': 

url_for('static', filename='style.css') 

File harus disimpan di sistem file sebagai static/style.csss

# Rendering Templates

Membuat HTML dari dalam Python tidaklah menyenangkan, dan sebenarnya cukup rumit karena kita harus melakukan HTML escaping sendiri untuk menjaga keamanan aplikasi. Karena itu Flask mengkonfigurasi template engine Jinja2 untuk kita  secara otomatis.

<b> Untuk merender template kita bisa menggunakan metode render_template()

```
from flask import render_template 
@app.route('/hello/') 
@app.route('/hello/<name>') 
def hello(name=None): 
    return render_template('hello.html', name=name)
```

<b> Flask akan mencari templates di folder templates .Jadi, jika aplikasi kita adalah sebuah modul, folder ini berada di sebelah modul tersebut, jika itu sebuah paket, itu sebenarnya ada di dalam package kita: </b>

Case 1: a module:
```
/application.py
/template
	/hello.html
```

Case 2: a package:
```
/application
	/__init__.py
	/template
		/hello.html
```

<b> Untuk template, kita dapat menggunakan template Jinja2.

Berikut ini contoh template:

```
<!doctype html>
<title>Hello from flask</title>
(% if name %)
    <h1>Hello {{ name }}</h1>
(% else %)
    <h1>Hello, World!</h1>
(% endif %)
```

# Make Our Own Flask

Ada beberapa hal yang perlu kita kumpulkan untuk aplikasi web. Dua yang pertama adalah: 
- Kode Python yang akan memuat model kita, mendapatkan input pengguna dari formulir web, melakukan prediksi, mengembalikan hasil. 
- Template HTML yang diisi dengan render. Ini memungkinkan pengguna untuk memasukkan data mereka sendiri dan akan mempresentasikan hasilnya.

Aplikasi kita pada awalnya akan memiliki struktur seperti ini:
```
H8Deployment/
    |-- model/
    |   |-- model_classifier.pkl
    |-- static
    |   |-- css
    |       |--style.css
    |-- templates/
    |   |-- main.html
    |-- app.py
```

### app.py 

Ini adalah inti dari aplikasi web kita. app.py juga yang akan dijalankan di server Heroku, mengirimkan halaman web dan memproses masukan dari pengguna.

In [ ]:
import flask 
app = flask.Flask(__name__, template_folder='templates') 

@app.route('/') 
def main(): 
    return(flask.render_template('main.html')) 
if __name__ == '__main__': 
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


### main.html

```
<!DOCTYPE html> 
<html lang="en"> 
<head> 
<meta charset="UTF-8"> 
<meta name="viewport" content="width=device-width, initial-scale=1.0"> 
<title>H8 Model Deployment</title> 
</head> 
<body> 
<h1>Welcome to My First Flask Apps!</h1> 
</body> 
</html>
```

## Running the test app

Untuk memulai flask di komputer lokal kita, pastikan kita berada di folder H8Deployment dan jalankan perintah: python app.py.

## Editing app.py to load the model

In [ ]:
import numpy as np 
import pickle 

model = pickle.load(open('model_classifier.pkl', 'rb'))

## Redirecting the API to predict the result

In [ ]:
@app.route('/predict',methods=['POST']) 
def predict(): 
    ''' For rendering results on HTML GUI ''' 
    int_features = [int(x) for x in flask.request.form.values()] 
    final_features = [np.array(int_features)] 
    prediction = model.predict(final_features) 
    
    output = {0: 'not placed', 1: 'placed'} 
    
    return flask.render_template('main.html', prediction_text='Student must be {} to workplace'.format(output[prediction[0]]))

Tambahkan debug=True di app.run.

In [ ]:
if __name__ == "__main__": 
    app.run(debug=True)

# Editing the main.html template

## The form 

```
<form action="{{ url_for('predict') }}" method="post">
```

Atribut action memberi tahu flask route mana harus dipanggil ketika form disubmit. Dalam aplikasi sederhana ini,kita hanya menginginkan main function.Metode POST  memberi tahu fungsi itu bahwa ia harus mengharapkan input dan setelahnya memprosesnya. 

## Some input checks

Elemen `<input>` dari form memiliki atribut required, untuk menjadikannya wajib.

```
<input type="text" name="ssc" placeholder="Secondary Education" required="required" /> 
<input type="text" name="hsc" placeholder="Highschool" required="required" /> 
<input type="text" name="degree" placeholder="Degree" required="required" /> 
<input type="text" name="etest" placeholder="Interview Score" required="required" /> 

<button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
```

## The results

file main.html kita akan terlihat seperti ini

```
<!DOCTYPE html> 
<html lang="en"> 
<head> 
<meta charset="UTF-8"> 
<meta name="viewport" content="width=device-width, initial-scale=1.0"> 
<title>H8 Model Deployment</title> 
</head> 
<body> 
<div class="form"> 
<h1>Hacktiv8 Student Placement Classifier</h1> 

<form action="{{ url_for('predict') }}" method="post"> 
<input type="text" name="ssc" placeholder="Secondary Education" required="required" /> 
<input type="text" name="hsc" placeholder="Highschool" required="required" /> 
<input type="text" name="degree" placeholder="Degree" required="required" /> 
<input type="text" name="etest" placeholder="Interview Score" required="required" /> 

<button type="submit" class="btn btn-primary btn-block btn-large">Predict</button> </form> 

<p class="message">{{ prediction_text }}</p> 
</div> 
</body> 
</html>
```

Restart Server Flask menggunakan python app.py dan uji masukan dan keluaran prediksi.

## Simple CSS

Tambahkan ```<link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}">``` pada main.html jadi file main.html kita dapat menemukan file styling. 

Pada style.css file, tambahkan beberapa kode:

```
@import url(https://fonts.googleapis.com/css?family=Roboto:300); 

.form { 
    position: relative; 
    z-index: 1; 
    background: #FFFFFF; 
    max-width: 360px; 
    margin: 0 auto 100px; 
    padding: 45px; 
    text-align: center; 
    box-shadow: 0 0 20px 0 rgba(0, 0, 0, 0.2), 0 5px 5px 0 rgba(0, 0, 0, 0.24); 
} 

.form input { 
    font-family: "Roboto", sans-serif; 
    outline: 0; 
    background: #f2f2f2; 
    width: 100%; 
    border: 0; 
    margin: 0 0 15px; 
    padding: 15px; 
    box-sizing: border-box; 
    font-size: 14px; 
} 

.form button { 
    font-family: "Roboto", sans-serif; 
    text-transform: uppercase; 
    outline: 0; 
    background: #F19023; 
    width: 100%; 
    border: 0; 
    padding: 15px; 
    color: #FFFFFF; 
    font-size: 14px; c
    ursor: pointer; 
} 

.form button:hover,.form button:active,.form button:focus { 
    background: #c5761c; 
} 
            
.form .message { 
    margin: 15px 0 0; 
    color: #b3b3b3; 
    font-size: 12px; 
} 

body { 
    background: #F19023; 
    font-family: "Roboto", sans-serif; 
}
```

Sekarang reload browser web kita.

# Ready for Deployment


## Setup

Pada langkah ini kita akan menginstal dan menjalankan Heroku Command Line Interface (CLI).Kita menggunakan CLI untuk manage dan scale aplikasi, menyediakan add-on, melihat log aplikasi, dan menjalankan aplikasi kita secara lokal.

1. Gunakan perintah <b> heroku login</b>  untuk login ke Heroku CLI.
2. Buat aplikasi di Heroku, yang mempersiapkan Heroku untuk menerima source code:<b> heroku create
3. Secara default, command tersebut akan membuat aplikasi dengan nama acak. Jika kita ingin memilih nama aplikasi sendiri, cukup pass sebagai argumen. Sebagai contoh: <b> heroku create h8-model-christian

Saat kita create app seperti diatas, git remote (disebut heroku) ) juga dibuat dan dikaitkan dengan repositori git lokal kita.


# Prepare the app

Buat `.gitignore` file to menghiraukan file generated seperti `__pycache__/` atau  `ipynb_checkpoints/`

Dalam folder H8Deployment , jalankan command berikut untuk membuat repository baru: <b> git init

### Create the required Heroku files

Kita akan membutuhkan dua file, ditempatkan di dalam folder H8Deployment.

<b> 1. requirements.txt </b>

File ini memberi tahu Heroku packages mana yang harus dipasang untuk aplikasi web kita. Akan terlihat seperti ini:

```
flask
numpy
sklearn
gunicorn
```

<b> 1. Procfile </b>

File ini memberi tahu Heroku jenis aplikasi yang kita jalankan dan cara menyajikannya kepada pengguna. File ini hanya berisi satu baris dan akan terlihat seperti ini: <b> web: gunicorn app:app

Folder aplikasi web kita sekarang akan terlihat seperti ini:

```
H8Deployment/
    |-- model/
    |   |-- model_classifier.pkl
    |-- static
    |   |-- css
    |       |--style.css
    |-- templates/
    |   |-- main.html
    |-- app.py
    |-- Procfile
    |-- requirements.txt
```

## Add files to repository

Saat berada di folder H8Deployment  , gunakan perintah berikut untuk menambahkan semua file aplikasi web ke repositori git:
```
git add .
git commit -m "First commit!"
```

# Set the remote destination for pushing from git to Heroku

Perintah ini memudahkan untuk push aplikasi web lokal ke Heroku, menggunakan git

Ex: <b> heroku git:remote -a h8-model-christian </b>

# Deploy the app

Sekarang deploy kode kita:

<b> git push heroku master

Aplikasi sekarang dideploy. Pastikan setidaknya satu instance aplikasi sedang berjalan:
    
<b> heroku ps:scale web=1

Sekarang kunjungi aplikasi di URL yang dihasilkan oleh nama aplikasinya. Sebagai shortcut praktis, kita dapat membuka situs web sebagai berikut:

<b> heroku open